In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import thunder as td
%matplotlib inline

In [5]:
def dff(v):
    import numpy as np
    baseline=np.percentile(v, 10)
    return (v - baseline) / (baseline + .1)

In [2]:
paths = {}
paths['base'] =  '/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20161019/fish3/20161019_3_1_GFAPGCaMP_6dpf_dark_0gain_20161019_201709/'
paths['raw']= paths['base'] + 'raw/'
paths['ep'] = '/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20161019/fish3/20161019_3_1_GFAPGCaMP_6dpf_dark_0gain_20161019_201709/ephys/data.mat'

fnames = glob(paths['raw'] + 'TM*')
fnames.sort()
t_range = slice(800, 1000)
fnames = fnames[t_range]
print(len(fnames))

200


In [7]:
%%time
ims = td.images.fromlist(fnames, accessor=im_loader, engine=sc, npartitions=len(fnames)).median_filter(size=(1,3,3))
dff_ims = ims.map_as_series(dff)

CPU times: user 36.4 s, sys: 504 ms, total: 36.9 s
Wall time: 45.7 s


In [8]:
%%time
local = dff_ims.map(lambda v: v.max(0)).toarray()

CPU times: user 5.24 s, sys: 2.1 s, total: 7.34 s
Wall time: 1min 21s


In [9]:
from skimage.io import imsave
imsave('/groups/ahrens/ahrenslab/davis/tmp/mov_test.tif', local.astype('float32'), compress=1)